In [38]:
#Importing packages
from selenium import webdriver
import pandas as pd

## Scraper
First of all, don't run it!, it took me hours to scrape all the data (It should not have taken so long but I don't know why).

Secondly, I scraped around 7500 most recent pieces of comment instead of 5000 because the most recent 5000 comments are contributed by 68 users. With 7500 comments, we can get opioins from 130 users, which diversifys our data.

In [16]:
%%time

#change directory where you put your chrome driver
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome("chromedriver", options=options)

all_comments = pd.DataFrame(columns = ['Date','user_id', 'comments']) 

x = 284
pages_scraped = 434
driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p{}'.format(x))

while x <= pages_scraped:
    print("page",x)
    
    if x > 1:
        driver.find_element_by_link_text("{}".format(x)).click()
        driver.implicitly_wait(3)
    
    ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
    comment_ids = []
    for i in ids:
        comment_ids.append(i.get_attribute('id'))

    for d in comment_ids:

        #Extract dates from for each user on a page
        user_date = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
        date = user_date.get_attribute('title')

        #Extract user ids from each user on a page
        userid_element = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[2]/div[1]/span[1]/a[2]')[0]
        userid = userid_element.text

        #Extract Message for each user on a page
        if driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/p'):
            user_message = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/p')[0]
            comment = user_message.text
        elif driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/ul/li'):
            user_message = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/ul/li')[0]
            comment = user_message.text
        elif driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]'):
            user_message = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]')[0]
            comment = user_message.text
        else: 
            comment = None
        
        #Adding date, userid and comment for each user in a dataframe    
        all_comments.loc[len(all_comments)] = [date,userid,comment]
        
    x += 1

In [69]:
# all_comments.to_csv("7500_comments_edmunds.csv", sep="\t")

## Check the data
We have 7506 rows of comments from 2012-8-10 to 2019-8-21, comtibuted by 130 users.

In [55]:
df = pd.read_csv("7500_comments_edmunds.csv", sep="\t", parse_dates=['Date'])
df = df[['Date', 'user_id', 'comments']]
df = df.dropna()

In [57]:
df.shape

(7506, 3)

In [58]:
df['Date'].max()

Timestamp('2019-08-21 02:25:00')

In [59]:
df['Date'].min()

Timestamp('2012-08-10 01:45:00')

In [60]:
len(df['user_id'].value_counts())

130

### Preprocess comments

In [89]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from itertools import chain
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\spong\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [140]:
%%time
# tokenization
df['tokens'] = df['comments'].map(nltk.word_tokenize)
df['tokens'] = df['tokens'].map(lambda x: set(x))

# lemmatization
lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df ['tokens'].map(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['lemmatized'] = df['lemmatized'].map(lambda x: set(x))

# remove stop words
stop_words = set(stopwords.words('english'))
df['processed'] = df['lemmatized'].map(lambda x: [word for word in x if word.lower() not in stop_words])

# remove punctuations
punc = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~..."
df['processed'] = df['processed'].map(lambda x: [word for word in x if word.lower() not in punc])

Wall time: 7.22 s


In [141]:
# remove some other stuff
others = ["''", "``", "n't", "l", "oh", "lol", "'m", "'s"]
df['processed'] = df['processed'].map(lambda x: [word for word in x if word.lower() not in others])

In [142]:
pd.set_option('display.max_rows', 500)
all_words = pd.Series(chain(*list(df['processed'])))
all_words.value_counts().head(150)

car            3225
wa             2298
said           2047
like           1985
would          1894
one            1827
ha             1719
get            1633
BMW            1528
think          1456
new            1414
year           1327
time           1160
even           1108
much           1086
good           1086
know           1065
really          992
doe             983
make            962
price           949
Audi            946
could           935
drive           924
well            897
better          889
still           882
go              881
dealer          863
thing           846
see             833
say             810
want            809
look            794
people          781
also            776
've             775
way             773
3               741
back            715
engine          695
going           693
mile            692
lot             687
model           686
take            651
Acura           650
need            637
'd              632
many            628
